# 基于MindSpore实现多分类损失函数
本小节主要介绍多分类损失函数的原理，并使用MIndspore实现。

## 1、实验目的
- 了解多分类损失函数原理。
- 掌握如何使用MIndspore实现多分类损失函数。


## 2、多分类损失函数原理介绍
二分类损失函数中定义了一个简单的平均绝对误差损失函数MAELoss，但许多深度学习应用的数据集较复杂，如目标检测网络Faster R-CNN的数据中就包含多个标签，而非简单的一条数据对应一个标签，这时损失函数的定义和使用略有不同。  
针对本实验中创建的多标签数据集，定义多标签损失函数MAELossForMultiLabel。
$$ loss\ 1=\frac{1}{m}\ \sum_{i=1}^{m}\left|y1_i-f\left(x_i\right)\right| $$
$$ loss\ 2=\frac{1}{m}\ \sum_{i=1}^{m}\left|y2_i-f\left(x_i\right)\right| $$
$$ loss=\frac{\left(loss1+loss2\right)}{2} $$
上式中，f(x)为样例标签的预测值，y1和y2为样例标签的真实值，$loss\ 1$为预测值与真实值y1之间距离的平均值，$loss\ 2$为预测值与真实值y2之间距离的平均值，loss为损失值$loss\ 1$与损失值$loss\ 2$平均值。  
在 MAELossForMultilabel中的construct方法的输入有三个，预测值base，真实值target1和target2，在construct中分别计算预测值与真实值target1、预测值与真实值target2之间的误差，将两误差取平均后作为最终的损失函数值。


## 3、实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 构建多标签数据集
Numpy模块主要用于数据的基本运算操作。MindSpore相关模块主要用于搭建网络、调用优化器、读取数据集和将数据集处理成网络的标准输入格式。  
数据集的两个标签分别由
$$y_1=2x+3+{noise}_1$$
$$y_2=2x+3+{noise}_2$$
生成。其中${noise}_1$和${noise}_2$为服从标准正态分布的随机值。


In [1]:
import numpy as np

from mindspore import dataset as ds
import mindspore.nn as nn
import mindspore as ms

ms.set_context(mode=ms.GRAPH_MODE, device_target='CPU')

# 生成带有两个标签的数据集
def get_multilabel_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        # noise1和noise2为服从标准正态分布的随机值
        noise1 = np.random.normal(0, 1)
        noise2 = np.random.normal(-1, 1)
        # 定义第一个标签
        y1 = x * w + b + noise1                   
        # 定义第二个标签
        y2 = x * w + b + noise2                   
        yield np.array([x]).astype(np.float32), np.array([y1]).astype(np.float32), np.array([y2]).astype(np.float32)

def create_multilabel_dataset(num_data, batch_size=16):
    dataset = ds.GeneratorDataset(list(get_multilabel_data(num_data)), column_names=['data', 'label1', 'label2'])
    # 每个batch有16个数据
    dataset = dataset.batch(batch_size) 
    return dataset

## 5、模型构建

### 5.1 多标签损失函数
定义多标签损失函数。

In [2]:
# 定义多标签损失函数
class MAELossForMultiLabel(nn.LossBase):
    def __init__(self, reduction="mean"):
        super(MAELossForMultiLabel, self).__init__(reduction)
        self.abs = ops.abs

    def construct(self, base, target1, target2):
        # 计算第一个标签的误差
        x1 = self.abs(base - target1)
        # 计算第二个标签的误差
        x2 = self.abs(base - target2)
        # 将两误差取平均后作为最终的损失函数值                           
        return (self.get_loss(x1) + self.get_loss(x2))/2        

### 5.2 定义损失函数
使用 Model 关联指定的前向网络、损失函数和优化器时，因 Model 内默认使用的 nn.WithLossCell 只接受两个输入： data 和 label ，故不适用于多标签场景。在多标签场景下，若想使用 Model 进行模型训练，则需事先把前向网络与多标签损失函数关联起来，即自定义损失网络。

In [3]:
# 自定义损失网络
class CustomWithLossCell(nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(CustomWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn

    def construct(self, data, label1, label2):
        output = self._backbone(data)
        return self._loss_fn(output, label1, label2)

### 5.3 定义网络模型
定义线性回归网络。

In [4]:
from mindspore.common.initializer import Normal
import mindspore.ops as ops
from mindspore.train import LossMonitor
# 定义线性回归网络
class LinearNet(nn.Cell):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)

## 6、模型训练
定义多标签损失函数、损失网络和优化器，并开始模型的训练。

In [5]:
ds_train = create_multilabel_dataset(num_data=160)
net = LinearNet()
# 定义多标签损失函数
loss = MAELossForMultiLabel()
# 定义损失网络，连接前向网络和多标签损失函数
loss_net = CustomWithLossCell(net, loss)
# 定义优化器
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)
# 定义Model，多标签场景下Model无需指定损失函数
model = ms.train.Model(network=loss_net, optimizer=opt)

model.train(epoch=10, train_dataset=ds_train, callbacks=[LossMonitor(1)])

epoch: 1 step: 1, loss is 11.168766021728516
epoch: 1 step: 2, loss is 9.674881935119629
epoch: 1 step: 3, loss is 7.414841651916504
epoch: 1 step: 4, loss is 8.943695068359375
epoch: 1 step: 5, loss is 8.155509948730469
epoch: 1 step: 6, loss is 10.243484497070312
epoch: 1 step: 7, loss is 7.751708984375
epoch: 1 step: 8, loss is 8.45167350769043
epoch: 1 step: 9, loss is 6.604907989501953
epoch: 1 step: 10, loss is 5.796879768371582
epoch: 2 step: 1, loss is 7.4184675216674805
epoch: 2 step: 2, loss is 4.518697738647461
epoch: 2 step: 3, loss is 2.9617834091186523
epoch: 2 step: 4, loss is 2.656360626220703
epoch: 2 step: 5, loss is 3.38094425201416
epoch: 2 step: 6, loss is 2.7276973724365234
epoch: 2 step: 7, loss is 2.4612107276916504
epoch: 2 step: 8, loss is 2.5444395542144775
epoch: 2 step: 9, loss is 2.829113006591797
epoch: 2 step: 10, loss is 2.8429994583129883
epoch: 3 step: 1, loss is 2.5914132595062256
epoch: 3 step: 2, loss is 2.4165596961975098
epoch: 3 step: 3, loss is

## 7、模型预测
使用模型进行预测。

In [6]:
from mindspore.train import Model
from mindspore import Tensor

model_predict = Model(net,loss_net,opt,metrics={"loss"})
# 生成测试数据
w=2.0
b=3.0
x = np.random.uniform(-10.0, 10.0, (1,1))
x1 = np.array([x]).astype(np.float32)
# 定义第一个标签
true_result1 = x * w + b      
# 定义第二个标签
true_result2 = x * w + b            
print('data:' + '%s'%x)
# 模型测试
test_result = model_predict.predict(Tensor(x1))
print('predict result:' + '%s'%test_result)
print('true result1:' + '%s'%true_result1)
print('true result2:' + '%s'%true_result2)

data:[[-1.31870369]]
predict result:[[[-0.08267879]]]
true result1:[[0.36259261]]
true result2:[[0.36259261]]
